In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
load_dotenv()

groq_api_key= os.getenv("GROQ_API_KEY")

## Building A Chatbot

In [ ]:
llm=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)

In [ ]:
llm.invoke([HumanMessage(content="Hi! My name is Charles")])

In [ ]:
messages=[
    HumanMessage(content="Hi! My name is Charles"),
    AIMessage(content="Hi Charles, it's nice to meet you! What can I do for you today?"),
    HumanMessage(content="What's my name")
]

In [ ]:
llm.invoke(messages)

#### Message history

In [ ]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(llm, get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat2"}}

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi! My name is Charles")],
    config=config
)

In [ ]:
print(response.content)

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

#### change the config --> session id

In [ ]:
config1={"configurable":{"session_id":"chat2"}}

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)
response.content

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, my name is John.")],
    config=config1
)
response.content

#### Prompt Templates

In [48]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI assistant. Answer all the questions to the best of your ability in {language}. If you don't an answer then say you don't know."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|llm

In [43]:
chain.invoke({"messages":[HumanMessage(content="Hi, my name is Charles")]})

AIMessage(content="Hello Charles, it's nice to meet you! \n\nI'm ready for your questions. Ask away!  😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 48, 'total_tokens': 78, 'completion_time': 0.054545455, 'prompt_time': 0.000605418, 'queue_time': 0.014031332, 'total_time': 0.055150873}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d20681a9-77ae-4784-b6df-c2925ff491d1-0', usage_metadata={'input_tokens': 48, 'output_tokens': 30, 'total_tokens': 78})

In [44]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [45]:
config1={"configurable":{"session_id":"chat3"}}

In [47]:
response=with_message_history.invoke(
        [HumanMessage(content="Hello, my name is Charles.")],
    config=config
)
response

AIMessage(content="Hello Charles, it's nice to meet you!  How can I assist you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 187, 'total_tokens': 211, 'completion_time': 0.043636364, 'prompt_time': 0.005491473, 'queue_time': 0.009258386, 'total_time': 0.049127837}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-23d4468a-85d7-4056-a153-574787420aea-0', usage_metadata={'input_tokens': 187, 'output_tokens': 24, 'total_tokens': 211})

In [49]:
response= chain.invoke({
        "messages":[HumanMessage(content="Hello, my name is Charles.")],"language":"French"})
response.content


"Bonjour Charles, enchanté de faire votre connaissance !  Comment puis-je vous aider aujourd'hui ? \n"

In [50]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [52]:
config={"configurable":{"session_id":"chat4"}}

In [54]:
response=with_message_history.invoke(
   {"messages":[HumanMessage(content="Hello, my name is Charles.")],"language":"French"},
   config=config
)
response.content

"Bonjour Charles, enchanté de te rencontrer ! Comment puis-je t'aider aujourd'hui ? 😊 \n"

In [55]:
response=with_message_history.invoke(
   {"messages":[HumanMessage(content="What's my name?.")],"language":"English"},
   config=config
)
response.content

'Your name is Charles. 😊  \n\n'